# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
!pip install ultralytics
!pip install opencv-python

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [2]:
import cv2

from ultralytics import YOLO, solutions


model = YOLO('yolov8s-obb.pt')

In [3]:
model = YOLO('/content/YOLO_Model.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [4]:
cap = cv2.VideoCapture("/content/YOLO_Video.mp4")

#Source: https://docs.ultralytics.com/guides/object-counting/#real-world-applications

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [5]:
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

#Source: https://docs.ultralytics.com/guides/object-counting/#real-world-applications

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [6]:
video_writer = cv2.VideoWriter("object_counting_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

#Source: https://docs.ultralytics.com/guides/object-counting/#real-world-applications

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [8]:
from shapely.geometry import Polygon

current_region = None
counting_regions = [
    {
        "name": "YOLOv8 Rectangle Region",
        "polygon": Polygon([(1200, 550), (1400, 650), (1400, 550), (1200, 650)]),  # Polygon points
        "counts": 0,
        "dragging": False,
        "region_color": (37, 255, 225),  # BGR Value
        "text_color": (0, 0, 0),  # Region Text Color
    }
]

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [ ]:
# Iterate over video frames
from ultralytics.utils.plotting import Annotator, colors
from collections import defaultdict
from shapely.geometry.point import Point
import numpy as np

vid_frame_count=0

#Select classes
classes = [9,10]
# Extract classes names
names = model.model.names
line_thickness = 1
region_thickness= 1
track_thickness = 1

track_history = defaultdict(list)

names = model.names

while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        vid_frame_count += 1

        annotator = Annotator(frame, 2, names)
        # Extract the results
        tracks = model.track(frame, persist=True)

        if tracks[0].boxes.id is not None:
            boxes = tracks[0].boxes.xyxy.cpu()
            clss = tracks[0].boxes.cls.cpu().tolist()
            track_ids = tracks[0].boxes.id.int().cpu().tolist()

        # Extract tracks
        for box, track_id, cls in zip(boxes, track_ids, clss):
          annotator.box_label(box, label=f"{names[cls]}#{track_id}", color=colors(int(track_id), True))
          bbox_center = (box[0] + box[2]) / 2, (box[1] + box[3]) / 2  # Bbox center


          # Draw Tracks
          track_line = track_history[track_id]
          track_line.append((float((box[0] + box[2]) / 2), float((box[1] + box[3]) / 2)))
          if len(track_line) > 30:
             track_line.pop(0)

          # Draw track trails

          annotator.draw_centroid_and_tracks(
                track_line,
                color= colors(int(track_id), True),
                track_thickness= 2,
                )

          prev_position = track_history[track_id][-2] if len(track_history[track_id]) > 1 else None

        # Check if detection inside region
          for region in counting_regions:
             if region["polygon"].contains(Point((bbox_center[0], bbox_center[1]))):
              region["counts"] += 1

        # Draw regions (Polygons/Rectangles)
          for region in counting_regions:
            region_label = str(region["counts"])
            region_color = region["region_color"]
            region_text_color = region["text_color"]

            polygon_coords = np.array(region["polygon"].exterior.coords, dtype=np.int32)
            centroid_x, centroid_y = int(region["polygon"].centroid.x), int(region["polygon"].centroid.y)

            text_size, _ = cv2.getTextSize(
                region_label, cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.7, thickness=line_thickness
            )
            text_x = centroid_x - text_size[0] // 2
            text_y = centroid_y + text_size[1] // 2

            cv2.rectangle(
                frame,
                (text_x - 5, text_y - text_size[1] - 5),
                (text_x + text_size[0] + 5, text_y + 5),
                region_color,
                -1,
            )
            cv2.putText(
                frame, region_label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, region_text_color, line_thickness
            )
            cv2.polylines(frame, [polygon_coords], isClosed=True, color=region_color, thickness=region_thickness)


        video_writer.write(frame)


        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

del vid_frame_count
video_writer.release()
cap.release()
cv2.destroyAllWindows()

#Source: https://github.com/ultralytics/ultralytics/blob/main/examples/YOLOv8-Region-Counter/yolov8_region_counter.py +
# Source https://github.com/ultralytics/ultralytics/blob/main/ultralytics/solutions/object_counter.py


0: 384x640 11 cars, 3582.6ms
Speed: 3.9ms preprocess, 3582.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 2293.5ms
Speed: 3.6ms preprocess, 2293.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2272.8ms
Speed: 3.4ms preprocess, 2272.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 2279.4ms
Speed: 3.2ms preprocess, 2279.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 2307.5ms
Speed: 3.7ms preprocess, 2307.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 3516.9ms
Speed: 3.7ms preprocess, 3516.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 2275.4ms
Speed: 3.3ms preprocess, 2275.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 2317.1ms
Speed: 3.6ms preprocess, 2317.1ms inference, 0.9ms postpro

## Save and Submit Your Work

Submit both the notebook and the output video